In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [9]:
from shapely.geometry import Point
def intpt_func(row):
    return Point(row['INTPTLON'], row['INTPTLAT'])

In [41]:
#loading LODES data

ham_lodes = pd.read_csv('./data/hamilton_lodes_2019.csv', dtype={"TRACTCE20_home":"string", "TRACTCE20_work":"string"})
ham_lodes.h_geocode = ham_lodes.h_geocode.apply(lambda x: int(x/1000))
ham_lodes.w_geocode = ham_lodes.w_geocode.apply(lambda x: int(x/1000))
ham_lodes.w_geocode = ham_lodes.w_geocode.astype(str)
ham_lodes.h_geocode = ham_lodes.h_geocode.astype(str)

#loading Hamilton county geodata
ham_cbg = pd.read_csv('./data/ham_cbg.csv')
ham_cbg['intpt'] = ham_cbg[['INTPTLAT', 'INTPTLON']].apply(lambda p: intpt_func(p), axis=1)
ham_cbg = gpd.GeoDataFrame(ham_cbg, geometry=gpd.GeoSeries.from_wkt(ham_cbg.geometry))
ham_cbg.GEOID = ham_cbg.GEOID.astype(str)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [264]:
#loading residential buildings
res_build = pd.read_csv('./data/ham_residential_buildings2.csv', index_col=0)
res_build = gpd.GeoDataFrame(res_build, geometry=gpd.GeoSeries.from_wkt(res_build.geometry))
res_build['location'] = res_build.geometry.apply(lambda p: [p.y, p.x])

#loading work buildings
com_build = pd.read_csv('./data/work_loc_poi_com_civ.csv', index_col=0)
com_build = gpd.GeoDataFrame(com_build, geometry=gpd.GeoSeries.from_wkt(com_build.geometry))
com_build['location'] = com_build.geometry.apply(lambda p: [p.y, p.x])
com_build = com_build.reset_index()
com_build.GEOID = com_build.GEOID.astype(str)

#loading all buildings (MS dataset)
ms_build = pd.read_csv('./data/ham_buildings_MS.csv')
ms_build = gpd.GeoDataFrame(ms_build, geometry=gpd.GeoSeries.from_wkt(ms_build.geo_centers))
ms_build.GEOID = ms_build.GEOID.astype(str)
ms_build['location'] = ms_build.geometry.apply(lambda p: [p.y, p.x])

In [160]:
ham_lodes = ham_lodes.groupby(['h_geocode', 'w_geocode']).agg(total_jobs=('total_jobs', sum)).reset_index().merge(ham_cbg[['GEOID', 'geometry']], left_on='h_geocode', right_on='GEOID').rename({'geometry':'home_geom'}, axis=1).drop('GEOID', axis=1).merge(ham_cbg[['GEOID', 'geometry']], left_on='w_geocode', right_on='GEOID').rename({'geometry':'work_geom'}, axis=1).drop('GEOID', axis=1).sort_values('total_jobs', ascending=False).reset_index(drop=True)
ham_lodes = gpd.GeoDataFrame(ham_lodes)

In [161]:
ham_lodes_homes = ham_lodes.drop('work_geom', axis=1).rename({'home_geom':'geometry'}, axis=1)
ham_lodes_work = ham_lodes.drop('home_geom', axis=1).rename({'work_geom':'geometry'}, axis=1)
ham_lodes_work.head()

,h_geocode,w_geocode,total_jobs,geometry
0,470650113214,470650004001,133,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
1,470650004001,470650004001,100,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
2,470650113231,470650004001,90,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
3,470650113142,470650004001,84,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
4,470650104353,470650004001,82,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."


In [63]:
res_build_cbg = res_build[['osmid', 'geometry']].sjoin(ham_lodes_homes)
com_build_cbg = com_build[['GEOID', 'geometry']].sjoin(ham_lodes_work)

In [72]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [174]:
#generating array of start and return times (in 15 min intervals)
from datetime  import datetime, timedelta
times_morning = [dt.strftime('%H:%M')+'am' for dt in 
       datetime_range(datetime(2016, 9, 1, 7), datetime(2016, 9, 1, 9, 10), 
       timedelta(minutes=15))]
times_evening = [dt.strftime('%H:%M')+'pm' for dt in 
       datetime_range(datetime(2016, 9, 1, 4), datetime(2016, 9, 1, 6, 6), 
       timedelta(minutes=15))]

In [96]:
res_build.GEOID = res_build.GEOID.astype(str)
com_build.GEOID = com_build.GEOID.astype(str)

In [266]:
import random
import tqdm
from tqdm.notebook import tqdm_notebook

prob_matrix = gpd.GeoDataFrame()
for idx, movement in tqdm_notebook(ham_lodes.iterrows(), total=ham_lodes.shape[0]):

    res = res_build[res_build.GEOID == movement.h_geocode].reset_index(drop=True)
    if res.empty:
        res = ms_build[ms_build.GEOID == movement.h_geocode].sample(n=movement.total_jobs, random_state=42).reset_index(drop=True)

    com = com_build[com_build.GEOID == movement.w_geocode].reset_index(drop=True)
    if com.empty:
        com = ms_build[ms_build.GEOID == movement.w_geocode].sample(n=movement.total_jobs, random_state=42).reset_index(drop=True)
        
    r = res
    c = com
   
    for job in range(movement.total_jobs):
     
        if c.empty:
            c = com
        elif r.empty:
            r = res

        rand_r = random.randrange(r.shape[0])
        rand_c = random.randrange(c.shape[0])
        r_df = r.iloc[rand_r]
        c_df = c.iloc[rand_c]
        r = r.drop([rand_r]).reset_index(drop=True)
        c = c.drop([rand_c]).reset_index(drop=True)

        time_slot1 = np.random.choice(times_morning, size=1, replace=True)
        time_slot2 = np.random.choice(times_evening, size=1, replace=True)

        temp = gpd.GeoDataFrame()

        temp.loc[job, 'h_geocode'] = movement.h_geocode
        temp.loc[job, 'w_geocode'] = movement.w_geocode
        temp.loc[job, 'total_jobs'] = movement.total_jobs
        temp.loc[job, 'home_loc_lat'] = r_df.location[0]
        temp.loc[job, 'home_loc_lon'] = r_df.location[1]
        temp.loc[job, 'work_loc_lat'] = c_df.location[0]
        temp.loc[job, 'work_loc_lon'] = c_df.location[1]
        temp.loc[job, 'go_time'] = time_slot1[0]
        temp.loc[job, 'return_time'] = time_slot2[0]

        prob_matrix = prob_matrix.append(temp, ignore_index=True)

  0%|          | 0/14282 [00:00<?, ?it/s]

In [ ]:
def func_home_pt(row):
    return Point(row.home_loc_lon, row.home_loc_lat)
def func_work_pt(row):
    return Point(row.work_loc_lon, row.work_loc_lat)

In [ ]:
prob_matrix['home_geom'] = prob_matrix[['home_loc_lat', 'home_loc_lon']].apply(lambda row: func_home_pt(row), axis=1)
prob_matrix['work_geom'] = prob_matrix[['work_loc_lat', 'work_loc_lon']].apply(lambda row: func_work_pt(row), axis=1)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [ ]:
prob_matrix.head()

,h_geocode,w_geocode,total_jobs,home_loc_lat,home_loc_lon,work_loc_lat,work_loc_lon,go_time,return_time,home_geom,work_geom
0,470650113214,470650004001,133.0,35.024889,-85.150876,35.049340,-85.290205,08:45am,05:15pm,POINT (-85.1508765 35.024889),POINT (-85.290205 35.04934)
1,470650113214,470650004001,133.0,35.028877,-85.143452,35.044792,-85.280033,07:00am,04:30pm,POINT (-85.14345240760954 35.02887652060261),POINT (-85.280033 35.044792)
2,470650113214,470650004001,133.0,35.028299,-85.156204,35.048659,-85.289930,07:45am,05:00pm,POINT (-85.1562035 35.028298500000005),POINT (-85.28993 35.048659)
3,470650113214,470650004001,133.0,35.030310,-85.142181,35.048659,-85.289930,07:30am,04:15pm,POINT (-85.1421810823235 35.03030972249326),POINT (-85.28993 35.048659)
4,470650113214,470650004001,133.0,35.032467,-85.142892,35.048659,-85.289930,07:15am,05:45pm,POINT (-85.1428915 35.032467),POINT (-85.28993 35.048659)


## for Safegraph

In [282]:
sg = gpd.read_file('../Downloads/ham_sg_jan_mar_21_reduced_cols.csv')


In [301]:
import math
sg.home_cbg = sg.home_cbg.apply(lambda x: str(math.floor(float(x))))
sg.poi_cbg = sg.poi_cbg.apply(lambda x: str(math.floor(float(x))))
sg.frequency = sg.frequency.apply(lambda x: (math.floor(float(x))))
sg.visits_monday = sg.visits_monday.astype(float) 
sg.visits_tuesday = sg.visits_tuesday.astype(float) 
sg.visits_wednesday = sg.visits_wednesday.astype(float) 
sg.visits_thursday = sg.visits_thursday.astype(float) 
sg.visits_friday = sg.visits_friday.astype(float) 
sg.visits_saturday = sg.visits_saturday.astype(float) 
sg.visits_sunday = sg.visits_sunday.astype(float) 

In [293]:
type(sg.iloc[0].frequency)

numpy.int64

In [303]:
sg.visits_monday.sum()

9468382.0

In [305]:
sg = sg.groupby(['home_cbg', 'poi_cbg']).agg(frequency=('frequency', sum), visits_monday=('visits_monday', sum), visits_tuesday=('visits_tuesday', sum), visits_wednesday=('visits_wednesday', sum), visits_thursday=('visits_thursday', sum), visits_friday=('visits_friday', sum), visits_saturday=('visits_saturday', sum), visits_sunday=('visits_sunday', sum) ).reset_index()

In [306]:
pd.set_option('display.max_columns', None)
sg.head()

,home_cbg,poi_cbg,frequency,visits_monday,visits_tuesday,visits_wednesday,visits_thursday,visits_friday,visits_saturday,visits_sunday
0,470650004001,470650004001,601,9152.0,10525.0,10742.0,10662.0,9263.0,3826.0,3736.0
1,470650004001,470650004002,12,27.0,32.0,36.0,18.0,23.0,12.0,10.0
2,470650004001,470650006001,50,467.0,393.0,403.0,415.0,520.0,483.0,557.0
3,470650004001,470650006002,4,10.0,10.0,13.0,7.0,4.0,8.0,6.0
4,470650004001,470650006003,52,738.0,851.0,665.0,875.0,792.0,363.0,214.0


In [310]:
import random
import tqdm
from tqdm.notebook import tqdm_notebook

prob_matrix_sg = gpd.GeoDataFrame()
for idx, movement in tqdm_notebook(sg.iterrows(), total=sg.shape[0]):

    res = res_build[res_build.GEOID == movement.home_cbg].reset_index(drop=True)
    if res.empty:
        res = ms_build[ms_build.GEOID == movement.home_cbg].sample(n=movement.frequency, random_state=42, replace=True).reset_index(drop=True)

    com = com_build[com_build.GEOID == movement.poi_cbg].reset_index(drop=True)
    if com.empty:
        com = ms_build[ms_build.GEOID == movement.poi_cbg].sample(n=movement.frequency, random_state=42, replace=True).reset_index(drop=True)
        
    r = res
    c = com
    # print(r)
    # print(movement.total_jobs)
    for freq in range(movement.frequency):
        
        # for iter in range(iters):
        if c.empty:
            c = com
        elif r.empty:
            r = res
        
        # print(c)
        # print(r.shape, c.shape)
        rand_r = random.randrange(r.shape[0])
        rand_c = random.randrange(c.shape[0])
        r_df = r.iloc[rand_r]
        c_df = c.iloc[rand_c]
        r = r.drop([rand_r]).reset_index(drop=True)
        c = c.drop([rand_c]).reset_index(drop=True)
        # print(r)
        # print(r_df)

        time_slot1 = np.random.choice(times_morning, size=1, replace=True)
        time_slot2 = np.random.choice(times_evening, size=1, replace=True)

        temp = gpd.GeoDataFrame()

        temp.loc[freq, 'home_cbg'] = movement.home_cbg
        temp.loc[freq, 'poi_cbg'] = movement.poi_cbg
        temp.loc[freq, 'frequency'] = movement.frequency
        temp.loc[freq, 'home_loc_lat'] = r_df.location[0]
        temp.loc[freq, 'home_loc_lon'] = r_df.location[1]
        temp.loc[freq, 'work_loc_lat'] = c_df.location[0]
        temp.loc[freq, 'work_loc_lon'] = c_df.location[1]
        temp.loc[freq, 'go_time'] = time_slot1[0]
        temp.loc[freq, 'return_time'] = time_slot2[0]

        # temp.loc[job, 'home_geom'] = Point([r_df.location[1], r_df.location[0]])
        prob_matrix_sg = prob_matrix_sg.append(temp, ignore_index=True)

# prob_matrix

  0%|          | 0/16561 [00:00<?, ?it/s]

In [ ]:
def func_home_pt(row):
    return Point(row.home_loc_lon, row.home_loc_lat)
def func_work_pt(row):
    return Point(row.work_loc_lon, row.work_loc_lat)

In [ ]:
prob_matrix_sg['home_geom'] = prob_matrix_sg[['home_loc_lat', 'home_loc_lon']].apply(lambda row: func_home_pt(row), axis=1)
prob_matrix_sg['work_geom'] = prob_matrix_sg[['work_loc_lat', 'work_loc_lon']].apply(lambda row: func_work_pt(row), axis=1)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [ ]:
prob_matrix_sg.head()

,h_geocode,w_geocode,total_jobs,home_loc_lat,home_loc_lon,work_loc_lat,work_loc_lon,go_time,return_time,home_geom,work_geom
0,470650113214,470650004001,133.0,35.024889,-85.150876,35.049340,-85.290205,08:45am,05:15pm,POINT (-85.1508765 35.024889),POINT (-85.290205 35.04934)
1,470650113214,470650004001,133.0,35.028877,-85.143452,35.044792,-85.280033,07:00am,04:30pm,POINT (-85.14345240760954 35.02887652060261),POINT (-85.280033 35.044792)
2,470650113214,470650004001,133.0,35.028299,-85.156204,35.048659,-85.289930,07:45am,05:00pm,POINT (-85.1562035 35.028298500000005),POINT (-85.28993 35.048659)
3,470650113214,470650004001,133.0,35.030310,-85.142181,35.048659,-85.289930,07:30am,04:15pm,POINT (-85.1421810823235 35.03030972249326),POINT (-85.28993 35.048659)
4,470650113214,470650004001,133.0,35.032467,-85.142892,35.048659,-85.289930,07:15am,05:45pm,POINT (-85.1428915 35.032467),POINT (-85.28993 35.048659)
